# Not a scaling law!

We will play with the [Transformers](https://huggingface.co/docs/transformers/en/index) and [Datasets](https://huggingface.co/docs/datasets/en/index) librairies of Hugging Face.

The question is: given a budget of compute, what is the impact of data scarcity?

To simplify, we will add some constraints:
- the number of parameters is fixed: you will fine-tune the distilGPT2 model.
- we fix the maximum sentence length at 64 tokens and assume the compute budget allows you to pass at most 100 times one sentence forward and backward through the model.

In [ ]:
import time
import datasets
from transformers import GPT2TokenizerFast, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

The code below loads the corpus

In [ ]:
t = GPT2TokenizerFast.from_pretrained('distilgpt2')
t.pad_token = t.eos_token
dc = DataCollatorForLanguageModeling(tokenizer=t, mlm=False)
d0 = datasets.load_dataset("wikitext","wikitext-2-v1")
dval = d0['validation']
dtrain = d0['train']

The code below constructs a training dataset and a validation dataset

In [ ]:
slen = 64
def tokenize(element):
    outputs = t(element["text"], truncation=True, max_length=slen, return_overflowing_tokens=True, return_length=True)
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == slen: input_batch.append(input_ids)
    return {"input_ids": input_batch}
dtrain = dtrain.map(tokenize, batched=True, remove_columns=dtrain.column_names)
dval = dval.map(tokenize, batched=True, remove_columns=dval.column_names)
print("training data",d0)

In [ ]:
dval = dval.select([i for i in range(10)])
print("validation data",dval)

Here is an example of code for finetuning with the [`Trainer` class](https://huggingface.co/docs/transformers/en/main_classes/trainer).

In [ ]:
d = dtrain.select([i for i in range(3)])
model = GPT2LMHeadModel.from_pretrained('distilgpt2')
trargs = TrainingArguments(".", do_train=True, num_train_epochs=5, per_device_train_batch_size=1, logging_steps=1, learning_rate=0.0001,
            per_device_eval_batch_size=1, eval_strategy="steps", eval_steps=1, report_to="none")
tr = Trainer(model=model, args=trargs, train_dataset=d, eval_dataset=dval, processing_class=t, data_collator=dc)
tr.train()

Given the constraints above, what experiment should you do in order to see the impact of data scarcity? 